In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

### **Importing & Cleaning the data**

In [2]:
df_orig = pd.read_csv('full_devices.csv', encoding_errors='ignore')

# We will keep an original copy of the
df = df_orig.copy()
print(df.shape)
df.head()

(124494, 12)


,date,device,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9
0,2015-01-01,S1F01085,0,215630672,56,0,52,6,407438,0,0,7
1,2015-01-01,S1F0166B,0,61370680,0,3,0,6,403174,0,0,0
2,2015-01-01,S1F01E6Y,0,173295968,0,0,0,12,237394,0,0,0
3,2015-01-01,S1F01JE0,0,79694024,0,0,0,6,410186,0,0,0
4,2015-01-01,S1F01R2B,0,135970480,0,0,0,15,313173,0,0,3


##### Drop Duplicates (if any)

In [3]:
print(f'len(df) before drop_duplicates:\t {len(df):,}')
print(f'len(df) after drop_duplicates:\t {len(df.drop_duplicates()):,}')
print(f'diff:\t\t\t\t {len(df)-len(df.drop_duplicates()):,}')

df.drop_duplicates(inplace=True)

len(df) before drop_duplicates:	 124,494
len(df) after drop_duplicates:	 124,493
diff:				 1


##### Null Values (if any)

In [4]:
df.isnull().sum()

date          0
device        0
failure       0
attribute1    0
attribute2    0
attribute3    0
attribute4    0
attribute5    0
attribute6    0
attribute7    0
attribute8    0
attribute9    0
dtype: int64

No null values were found.

##### Redundancy

Observing the data, I noticed that two specific columns might be identical:
- `attribute7` and `attribute8`

Let's validate and remove one of them if that is the case:

In [5]:
df_redundant = df.copy()
df_redundant['7_8_identical'] = df_redundant['attribute7'] == df_redundant['attribute8']
df_redundant['7_8_identical'].value_counts()

7_8_identical
True    124493
Name: count, dtype: int64

Indeed, both columns are identical, so let's remove one of them (`attribute8`)

In [6]:
df.drop(columns='attribute8', axis=1, inplace=True)
df.head()

,date,device,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute9
0,2015-01-01,S1F01085,0,215630672,56,0,52,6,407438,0,7
1,2015-01-01,S1F0166B,0,61370680,0,3,0,6,403174,0,0
2,2015-01-01,S1F01E6Y,0,173295968,0,0,0,12,237394,0,0
3,2015-01-01,S1F01JE0,0,79694024,0,0,0,6,410186,0,0
4,2015-01-01,S1F01R2B,0,135970480,0,0,0,15,313173,0,3


##### Dtypes

In [7]:
df.dtypes

date          object
device        object
failure        int64
attribute1     int64
attribute2     int64
attribute3     int64
attribute4     int64
attribute5     int64
attribute6     int64
attribute7     int64
attribute9     int64
dtype: object

In [8]:
# We will need to convert 'date' column to datetime format
df['date_dt'] = pd.to_datetime(df['date'])

In [9]:
print(df[['date','date_dt']].dtypes)
df[['date','date_dt']].head(3)

date               object
date_dt    datetime64[ns]
dtype: object


,date,date_dt
0,2015-01-01,2015-01-01
1,2015-01-01,2015-01-01
2,2015-01-01,2015-01-01


##### Describe Data

In [10]:
df.describe().round(2)

,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute9,date_dt
count,124493.00,1.244930e+05,124493.00,124493.00,124493.00,124493.00,124493.00,124493.00,124493.00,124493
mean,0.00,1.223875e+08,159.49,9.94,1.74,14.22,260173.03,0.29,12.45,2015-04-16 05:19:50.900692992
min,0.00,0.000000e+00,0.00,0.00,0.00,1.00,8.00,0.00,0.00,2015-01-01 00:00:00
25%,0.00,6.128346e+07,0.00,0.00,0.00,8.00,221452.00,0.00,0.00,2015-02-09 00:00:00
50%,0.00,1.227971e+08,0.00,0.00,0.00,10.00,249800.00,0.00,0.00,2015-03-27 00:00:00
75%,0.00,1.833091e+08,0.00,0.00,0.00,12.00,310266.00,0.00,0.00,2015-06-17 00:00:00
max,1.00,2.441405e+08,64968.00,24929.00,1666.00,98.00,689161.00,832.00,18701.00,2015-11-02 00:00:00
std,0.03,7.045934e+07,2179.67,185.75,22.91,15.94,99151.39,7.44,191.43,NaN


In [11]:
# FAILURE distribution
print(f'Number of records with failure: {len(df[df['failure']==1])} out of {len(df):,} records \t {round((len(df[df['failure']==1])/len(df))*100,2)}%')
print(f'Number of devices with failure: {len(df[df['failure']==1])} out of {df.device.nunique():,} devices \t {round((len(df[df['failure']==1])/df.device.nunique())*100,2)}%')

Number of records with failure: 106 out of 124,493 records 	 0.09%
Number of devices with failure: 106 out of 1,169 devices 	 9.07%




### **APPROACHING THE PROBLEM**

##### APPROACH #2
As of right now we are preparing the dataset aggregating all data per device.
- Our train data will be a random sample of devices (70% of all devices).
- Our test data will be the remaining (30%) devices.
- Here we would answer the question: Given today's data, what is the probability of the device failing TODAY.
- **ASSUMPTION**: We receive the current date's attributes before it fail and we want to predict if the device would fail at the end of the day.
    - I am not sure we can assume this...

##### APPROACH #2
We could also look at each datapoint, each event independently, and:
- Train the data with part (70%) of the devices that have failed and the scenarios that lead to failure.
- Test the data with the remaining (30%) of the devices that have failed and the probability of failure for each event.
- Here the prediction will be: what is the probability of the device failing in the next event.
    - This would answer our initial question: Should we apply maintenance to that device before it fails?
- **ISSUE**: No aggregation would be done and the number of events with failure is way too small for this approach (106 failures/126,493 records = 0.09% event failure rate)

##### APPROACH #3
We could aggregate the data, but instead of observing the most recent event, we:
- Aggregate the data up to the "second to last" event and have a new response variable: "will fail in next event"
- Our train data will be a random sample of devices (70% of all devices).
- Our test data will be the remaining (30%) devices.
- Here we would answer the question: Given today's data, what is the probability of the device failing TOMORROW.

**I believe that APPROACH #3 is the most adequate.**

Since there is not much information on what each attribute means, we will proceed to aggregate data per device and generate features from that point on.

REMEMBER: We will aggregate the data up to the second to last most recent event for each device.

For this we will need to: 
- remove the most recent record for each device,
- Aggregate the data,
- Add the second to last most recent event's features,
- Add the response variable from the most recent record (which we assume we will not receive when making the predictions).
    - The response variable will be named '`will_fail`'

In [12]:
# df with all most recent record for each device.
df_most_recent = (df[['device', 'date_dt']]
                  .groupby('device')
                  .agg(date_dt=('date_dt','max'))
                  .reset_index()
                  ).merge(df, how='left', on=['device', 'date_dt'])

# df with all records except most recent for each device
#df_except_most_recent = df[df[]]

df_most_recent.failure.value_counts()

failure
0    1068
1     101
Name: count, dtype: int64

Here we have some interesting infromation: there are 5 devices from the 106 total devices with failure that did not have their failures as their respective very last event.

Let's see which devices those are:

In [13]:
df_temp = df.merge(df_most_recent[['device', 'date_dt']].rename(columns={'device':'_device', 'date_dt':'_date_dt'}), 
                   left_on=['device', 'date_dt'], 
                   right_on=['_device', '_date_dt'], 
                   how="outer", 
                   indicator=True
                   ).query('_merge=="left_only"').drop(columns=['_merge', '_device', '_date_dt'], axis=1)

df_temp_clean = df_temp[df_temp['device'].isin(df_temp[df_temp['failure']==1].device.unique())].sort_values(['device', 'date_dt'])

print(df_temp_clean.device.unique().tolist())
print(f'number of rows these devices represent in df: {len(df[df['device'].isin(df_temp_clean.device.unique().tolist())])} out of {len(df):,}')


['S1F0GPFZ', 'S1F136J0', 'W1F0KCP2', 'W1F0M35B', 'W1F11ZG9']
number of rows these devices represent in df: 822 out of 124,493


We could simply eliminate these devices from out dataset, but because we have a small amount of failed cases, 
we will simply remove the records that are not interesting to us, that is, the events after event with failure.

For the records without failure (failure = 0) we will also remove the most recent record for which we are sure did not fail (otherwise we would be assuming that we know that the next event, which we don't have, would not fail).

With all that being said, we will: 
- use the cumsum aggregation and remove all records for which cumsum > 0 (for devices with failure)
- use 
- remove respective most recent record for devices without failure.
- add the response variable 'will_fail' = 1 for devices with failure, and 0 for devices without failure.
- we will then have two df's: one with `'will_fail'=1` and another with `'will_fail'=0`.
- We then proceed to merge these two df's together.

In [14]:
# Use the cumsum aggregation and remove all records for which cumsum > 0 (for devices with failure)

## isolate cases with failure
cases_with_failure = df[df['failure']==1].device.unique().tolist()
df_devices_failure = df[df['device'].isin(cases_with_failure)].copy()
## Create 'cumsum_failure'
df_devices_failure['cumsum_failure'] = df_devices_failure.sort_values(['device','date']).groupby('device')['failure'].cumsum()
## remove all records with 'cumsum_failure' > 0
df_devices_failure = df_devices_failure[df_devices_failure['cumsum_failure']==0].drop(columns='cumsum_failure', axis=1)
## add response variable: 'will_fail'
df_devices_failure['will_fail'] = 1

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

# Remove respective most recent record for devices without failure.

## isolate cases without failure
cases_without_failure = df[~df['device'].isin(cases_with_failure)].device.unique().tolist()
df_devices_nofailure = df[df['device'].isin(cases_without_failure)].copy()

## create temporary df with device and most recent date, and remove these rows from df
df_devices_nofailure = df_devices_nofailure.merge((df_devices_nofailure
                                                   .groupby('device')
                                                   .agg(most_recent_date=('date_dt','max'))
                                                   .reset_index()
                                                   .rename(columns={'device':'_device'})
                                                   ),
                                                  how='outer',
                                                  left_on=['device', 'date_dt'], 
                                                  right_on=['_device', 'most_recent_date'], 
                                                  indicator=True).query('_merge=="left_only"').drop(columns=['_merge', '_device', 'most_recent_date'], axis=1)
df_devices_nofailure['will_fail'] = 0

df_devices_nofailure.head()

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

df_clean = pd.concat([df_devices_nofailure, df_devices_failure], )

df_clean.head()

,date,device,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute9,date_dt,will_fail
0,2015-01-01,S1F01085,0,215630672,56,0,52,6,407438,0,7,2015-01-01,0
1,2015-01-02,S1F01085,0,1650864,56,0,52,6,407438,0,7,2015-01-02,0
2,2015-01-03,S1F01085,0,124017368,56,0,52,6,407438,0,7,2015-01-03,0
3,2015-01-04,S1F01085,0,128073224,56,0,52,6,407439,0,7,2015-01-04,0
4,2015-01-05,S1F01085,0,97393448,56,0,52,6,408114,0,7,2015-01-05,0


Now we can proceed to aggregate the data

### **Aggregating the data**

For each attribute we will create it's `mean`, `min`, `max`, `sum` and `nunique` version.

In [15]:
# First aggregate the attributes and date var
df_devices_att = (
    df_clean[['device', 'attribute1', 'attribute2', 'attribute3', 
              'attribute4', 'attribute5', 'attribute6', 'attribute7', #'attribute8'
              'attribute9', 'date_dt']]
       .groupby('device')
       .agg(['min','max', 'mean', 'nunique']) # , 'sum'])
       ).reset_index()

df_devices_att.columns = ['_'.join(col) for col in df_devices_att.columns.values]
df_devices_att.rename(columns={'device_':'device'}, inplace=True)

# Second, we generate the response variable for each device (makes no sense to aggregate 'will_fail' in min, max, mean, nunique)
df_devices_resp = (
    df_clean[['device', 'will_fail']]
    .groupby('device')
    .agg(will_fail=('will_fail','max'))
    )

df_devices = df_devices_resp.merge(df_devices_att, how='left', on='device')
# df_devices.head()
print(df_devices.columns)

Index(['device', 'will_fail', 'attribute1_min', 'attribute1_max',
       'attribute1_mean', 'attribute1_nunique', 'attribute2_min',
       'attribute2_max', 'attribute2_mean', 'attribute2_nunique',
       'attribute3_min', 'attribute3_max', 'attribute3_mean',
       'attribute3_nunique', 'attribute4_min', 'attribute4_max',
       'attribute4_mean', 'attribute4_nunique', 'attribute5_min',
       'attribute5_max', 'attribute5_mean', 'attribute5_nunique',
       'attribute6_min', 'attribute6_max', 'attribute6_mean',
       'attribute6_nunique', 'attribute7_min', 'attribute7_max',
       'attribute7_mean', 'attribute7_nunique', 'attribute9_min',
       'attribute9_max', 'attribute9_mean', 'attribute9_nunique',
       'date_dt_min', 'date_dt_max', 'date_dt_mean', 'date_dt_nunique'],
      dtype='object')


##### Most recent row's data

Once we have a trained model, we will receive new data for prediction which will include:
- Historical data on the previous days for a device,
- Data on the curent date, for which we are trying to make the prediction.

With that being said, the current (most recent) date's data is relevant for the prediction.

Let's include this data in our agregated dataframe (df_devices)

In [16]:
df_devices = df_devices.merge(df, how='left', left_on=['device','date_dt_max'], right_on=['device','date_dt'])

In [17]:
df_devices.columns

Index(['device', 'will_fail', 'attribute1_min', 'attribute1_max',
       'attribute1_mean', 'attribute1_nunique', 'attribute2_min',
       'attribute2_max', 'attribute2_mean', 'attribute2_nunique',
       'attribute3_min', 'attribute3_max', 'attribute3_mean',
       'attribute3_nunique', 'attribute4_min', 'attribute4_max',
       'attribute4_mean', 'attribute4_nunique', 'attribute5_min',
       'attribute5_max', 'attribute5_mean', 'attribute5_nunique',
       'attribute6_min', 'attribute6_max', 'attribute6_mean',
       'attribute6_nunique', 'attribute7_min', 'attribute7_max',
       'attribute7_mean', 'attribute7_nunique', 'attribute9_min',
       'attribute9_max', 'attribute9_mean', 'attribute9_nunique',
       'date_dt_min', 'date_dt_max', 'date_dt_mean', 'date_dt_nunique', 'date',
       'failure', 'attribute1', 'attribute2', 'attribute3', 'attribute4',
       'attribute5', 'attribute6', 'attribute7', 'attribute9', 'date_dt'],
      dtype='object')

We now have the aggregated data to engineer features

### **Feature engineering**

##### based on 'date_dt'

We will start by calculating the number of days we have data for each device (`date_dt_max-date_dt_min`).

This feature will be called: `num_days`

In [18]:
df_devices['num_days'] = (df_devices['date_dt_max']-df_devices['date_dt_min']).dt.days
print(df_devices[['date_dt_max', 'date_dt_min', 'num_days']].dtypes)
df_devices[['date_dt_max', 'date_dt_min', 'num_days']].head()

date_dt_max    datetime64[ns]
date_dt_min    datetime64[ns]
num_days                int64
dtype: object


,date_dt_max,date_dt_min,num_days
0,2015-01-05,2015-01-01,4
1,2015-05-10,2015-05-06,4
2,2015-01-05,2015-01-01,4
3,2015-02-16,2015-01-01,46
4,2015-01-05,2015-01-01,4


The most recent event we have happens in `date_dt_max`, so we will etract:
- `date_dt_max_day`: what day did the last device's event happen on.
- `date_dt_max_month`: what month did the last device's event happen on.
- `date_dt_max_weekday`: what weekday did the last device's event happen on. (0 = monday, 6 = sunday).

`date_dt_max_year`: we will ignore this feature for it is he same for all observations (2015)

In [19]:
# date_dt_max_day
df_devices['date_dt_max_day'] = df_devices['date_dt_max'].dt.day

# date_dt_max_month
df_devices['date_dt_max_month'] = df_devices['date_dt_max'].dt.month

# date_dt_max_weekday
df_devices['date_dt_max_weekday'] = df_devices['date_dt_max'].dt.weekday

# VALIDATE
print(df_devices[['date_dt_max_day', 'date_dt_max_month', 'date_dt_max_weekday']].describe().round(1))
df_devices[['date_dt_max', 'date_dt_max_day', 'date_dt_max_month', 'date_dt_max_weekday']].head()


       date_dt_max_day  date_dt_max_month  date_dt_max_weekday
count           1168.0             1168.0               1168.0
mean              13.0                4.3                  2.7
std                8.7                3.3                  2.4
min                1.0                1.0                  0.0
25%                5.0                1.0                  0.0
50%               11.0                3.0                  2.0
75%               21.0                8.0                  6.0
max               31.0               10.0                  6.0


,date_dt_max,date_dt_max_day,date_dt_max_month,date_dt_max_weekday
0,2015-01-05,5,1,0
1,2015-05-10,10,5,6
2,2015-01-05,5,1,0
3,2015-02-16,16,2,0
4,2015-01-05,5,1,0


In [20]:
df_devices.dtypes

device                         object
will_fail                       int64
attribute1_min                  int64
attribute1_max                  int64
attribute1_mean               float64
attribute1_nunique              int64
attribute2_min                  int64
attribute2_max                  int64
attribute2_mean               float64
attribute2_nunique              int64
attribute3_min                  int64
attribute3_max                  int64
attribute3_mean               float64
attribute3_nunique              int64
attribute4_min                  int64
attribute4_max                  int64
attribute4_mean               float64
attribute4_nunique              int64
attribute5_min                  int64
attribute5_max                  int64
attribute5_mean               float64
attribute5_nunique              int64
attribute6_min                  int64
attribute6_max                  int64
attribute6_mean               float64
attribute6_nunique              int64
attribute7_m

All in all, from the date columns, we have the features:
- `num_days`: (int64)
- `date_dt_min`: (datetime64[ns])
- `date_dt_max`: (datetime64[ns])
- `date_dt_mean`: (datetime64[ns])
- `date_dt_max_day`: (int32)
- `date_dt_max_month`: (int32)
- `date_dt_max_weekday`: (int32)


##### based on 'device'
I noticed that there are some possible categories we could extract from the `device` column.

In [21]:
# Extract the first three characters of the 'device' column
df_devices['device_0_3'] = df_devices['device'].str[0:3]
print(f'first 3 charcaters: {df_devices['device_0_3'].unique().tolist()}')

# Extract the first three characters of the 'device' column (more granular)
df_devices['device_0_4'] = df_devices['device'].str[0:4]
print(f'first 4 charcaters: {df_devices['device_0_4'].unique().tolist()}')

first 3 charcaters: ['S1F', 'W1F', 'Z1F']
first 4 charcaters: ['S1F0', 'S1F1', 'W1F0', 'W1F1', 'Z1F0', 'Z1F1', 'Z1F2']


Further on, we will apply encoding to these categorica variables. But before, out of curiosity, let's if there is any concentration of device failure in one of these categories:

In [23]:
# For first three characters ('device_0_3')
df_devices[['device_0_3','will_fail']].groupby('device_0_3').agg(num_devices=('device_0_3','count'),
                                                                 will_fail_num=('will_fail','sum'),
                                                                 will_fail_pct=('will_fail','mean')
                                                                 )

,num_devices,will_fail_num,will_fail_pct
device_0_3,,,
S1F,530,42,0.079245
W1F,419,42,0.100239
Z1F,219,22,0.100457


In [24]:
# For first four characters ('device_0_4')
df_devices[['device_0_4','will_fail']].groupby('device_0_4').agg(num_devices=('device_0_4','count'),
                                                                 will_fail_num=('will_fail','sum'),
                                                                 will_fail_pct=('will_fail','mean')
                                                                 ).sort_values('will_fail_pct', ascending=False)

,num_devices,will_fail_num,will_fail_pct
device_0_4,,,
Z1F1,67,9,0.134328
W1F1,137,15,0.109489
W1F0,282,27,0.095745
S1F0,391,36,0.092072
Z1F0,149,13,0.087248
S1F1,139,6,0.043165
Z1F2,3,0,0.000000


I believe we have created all possible relevant features with the data that we were given.

Before we continue we will create a function that applies all of that preprocessing as well as the encoding of the categorical variabels.

### **Preprocessing and Encoding**

In [27]:
def preprocessing_train(df):
    # -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
    # Drop duplicates
    print('1) Dropping duplicates...')
    df.drop_duplicates(inplace=True)
    print('Done!')
    print('-'*50)

    # -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
    # Convert 'date' column from object to datetime
    print('2) Converting \'date\' columns to datetime format...')
    df['date_dt'] = pd.to_datetime(df['date'])
    print('Done!')
    print('-'*50)

    # -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
    # Approach #3 data prep.
    print("Approach #3 data prep. started... ")

    # Use the cumsum aggregation and remove all records for which cumsum > 0 (for devices with failure)
    ## isolate cases with failure
    cases_with_failure = df[df['failure']==1].device.unique().tolist()
    df_devices_failure = df[df['device'].isin(cases_with_failure)].copy()
    ## Create 'cumsum_failure'
    df_devices_failure['cumsum_failure'] = df_devices_failure.sort_values(['device','date']).groupby('device')['failure'].cumsum()
    ## remove all records with 'cumsum_failure' > 0
    df_devices_failure = df_devices_failure[df_devices_failure['cumsum_failure']==0].drop(columns='cumsum_failure', axis=1)
    ## add response variable: 'will_fail'
    df_devices_failure['will_fail'] = 1

    # - - - - - - - - - - - - - - - -
    # Remove respective most recent record for devices without failure.
    ## isolate cases without failure
    cases_without_failure = df[~df['device'].isin(cases_with_failure)].device.unique().tolist()
    df_devices_nofailure = df[df['device'].isin(cases_without_failure)].copy()

    ## create temporary df with device and most recent date, and remove these rows from df
    df_devices_nofailure = df_devices_nofailure.merge((df_devices_nofailure
                                                    .groupby('device')
                                                    .agg(most_recent_date=('date_dt','max'))
                                                    .reset_index()
                                                    .rename(columns={'device':'_device'})
                                                    ),
                                                    how='outer',
                                                    left_on=['device', 'date_dt'], 
                                                    right_on=['_device', 'most_recent_date'], 
                                                    indicator=True).query('_merge=="left_only"').drop(columns=['_merge', '_device', 'most_recent_date'], axis=1)
    df_devices_nofailure['will_fail'] = 0
    
    # - - - - - - - - - - - - - - - -
    df_clean = pd.concat([df_devices_nofailure, df_devices_failure])

    print('Done!')
    print('-'*50)

    # -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
    # Aggregating the data
    print('3) Aggergating the data (groupby \'device\')...')

    # First aggregate the attributes and date var
    df_devices_att = (
        df_clean[['device', 'attribute1', 'attribute2', 'attribute3', 
                'attribute4', 'attribute5', 'attribute6', 'attribute7', #'attribute8'
                'attribute9', 'date_dt']]
        .groupby('device')
        .agg(['min','max', 'mean', 'nunique']) # , 'sum'])
        ).reset_index()

    df_devices_att.columns = ['_'.join(col) for col in df_devices_att.columns.values]
    df_devices_att.rename(columns={'device_':'device'}, inplace=True)

    # Second, we generate the response variable for each device (makes no sense to aggregate 'will_fail' in min, max, mean, nunique)
    df_devices_resp = (
        df_clean[['device', 'will_fail']]
        .groupby('device')
        .agg(will_fail=('will_fail','max'))
        )

    df_devices = df_devices_resp.merge(df_devices_att, how='left', on='device')
    # df_devices.head()
    print(df_devices.columns)
    
    print('Done!')
    print('-'*50)

    # -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
    # Feature Engineering
    print('4) Feature Engineering phase:')
    
    # 'num_days'
    print("creating 'num_days'...")
    df_devices['num_days'] = (df_devices['date_dt_max']-df_devices['date_dt_min']).dt.days
    print("'num_days' created!")

    # 'date_dt_max_day'
    print("creating 'date_dt_max_day'...")
    df_devices['date_dt_max_day'] = df_devices['date_dt_max'].dt.day
    print("'date_dt_max_day' created!")

    # 'date_dt_max_month'
    print("creating 'date_dt_max_month'...")
    df_devices['date_dt_max_month'] = df_devices['date_dt_max'].dt.month
    print("'date_dt_max_month' created!")

    # 'date_dt_max_weekday'
    print("creating 'date_dt_max_weekday'...")
    df_devices['date_dt_max_weekday'] = df_devices['date_dt_max'].dt.weekday
    print("'date_dt_max_weekday' created!")
    
    # 'device_0_4'
    print("creating 'device_0_4'...")
    df_devices['device_0_4'] = df_devices['device'].str[0:4]
    print("'device_0_4' created!")

    print('Feature Engineering phase done!')
    print('-'*50)

    # -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
    print('PREPROCESSING IS DONE!')
    return df_devices[[
        'device', 
        # 'date', 'failure', 'date_dt',
        'date_dt_min', 'date_dt_max', 'num_days', 'date_dt_mean',
        'date_dt_max_day', 'date_dt_max_month', 'date_dt_max_weekday',
        'device_0_4',
        'attribute1', 'attribute1_min', 'attribute1_max', 'attribute1_mean', 'attribute1_nuniq', 
        'attribute2', 'attribute2_min', 'attribute2_max', 'attribute2_mean', 'attribute2_nuniq', 
        'attribute3', 'attribute3_min', 'attribute3_max', 'attribute3_mean', 'attribute3_nuniq', 
        'attribute4', 'attribute4_min', 'attribute4_max', 'attribute4_mean', 'attribute4_nuniq', 
        'attribute5', 'attribute5_min', 'attribute5_max', 'attribute5_mean', 'attribute5_nuniq', 
        'attribute6', 'attribute6_min', 'attribute6_max', 'attribute6_mean', 'attribute6_nuniq', 
        'attribute7', 'attribute7_min', 'attribute7_max', 'attribute7_mean', 'attribute7_nuniq', 
        'attribute8', 'attribute8_min', 'attribute8_max', 'attribute8_mean', 'attribute8_nuniq', 
        'attribute9', 'attribute9_min', 'attribute9_max', 'attribute9_mean', 'attribute9_nuniq',
        'will_fail']]

    

- STOPPED HERE - STOPPED HERE - STOPPED HERE - STOPPED HERE - STOPPED HERE - STOPPED HERE - STOPPED HERE - STOPPED HERE - STOPPED HERE - STOPPED HERE 

### PENDING FORM HERE
- Create the preprocessing_test function (not involving the will_fail response variable of course)
- Create sample function.
- Correlation matrix and scatter matrix.
- Test out a couple of algorithms.


Index(['device', 'will_fail', 'attribute1_min', 'attribute1_max',
       'attribute1_mean', 'attribute1_nunique', 'attribute2_min',
       'attribute2_max', 'attribute2_mean', 'attribute2_nunique',
       'attribute3_min', 'attribute3_max', 'attribute3_mean',
       'attribute3_nunique', 'attribute4_min', 'attribute4_max',
       'attribute4_mean', 'attribute4_nunique', 'attribute5_min',
       'attribute5_max', 'attribute5_mean', 'attribute5_nunique',
       'attribute6_min', 'attribute6_max', 'attribute6_mean',
       'attribute6_nunique', 'attribute7_min', 'attribute7_max',
       'attribute7_mean', 'attribute7_nunique', 'attribute9_min',
       'attribute9_max', 'attribute9_mean', 'attribute9_nunique',
       'date_dt_min', 'date_dt_max', 'date_dt_mean', 'date_dt_nunique', 'date',
       'failure', 'attribute1', 'attribute2', 'attribute3', 'attribute4',
       'attribute5', 'attribute6', 'attribute7', 'attribute9', 'date_dt',
       'num_days', 'date_dt_max_day', 'date_dt_max_month

In [ ]:
(df.groupby('device').agg(failure_sum=('failure','sum')).reset_index()).groupby('failure_sum').agg(num_devices=('device','nunique'))

In [ ]:
# Percentage failed devices
print(f'{round((df_devices[df_devices['failed']==1].failed.sum()/len(df_devices))*100,2)}%')

df_devices.columns

In [ ]:
def correlation_matrix(df):
    try:
        correlation_matrix = df.corr()
        return correlation_matrix
    except KeyError:
        print(f"DataFrame not found.")
        return None
    
correlation_matrix(df_devices.drop(['device', 'date_min', 'date_max'], axis=1))

In [ ]:
def correlation_ratio(categorical_feature, numeric_feature):
    cats, freqs = np.unique(categorical_feature, return_counts=True)
    numeric_mean = np.mean(numeric_feature)
    sig_y_bar = 0
    for i in range(len(cats)):
        category_mean = np.mean(numeric_feature[categorical_feature == cats[i]])
        sig_y_bar += np.square(category_mean - numeric_mean) * freqs[i]
    sig_y = np.sum(np.square(numeric_feature - numeric_mean))
    statistic = np.sqrt(sig_y_bar / sig_y)
    return statistic

data = df_devices.drop(['device', 'date_min', 'date_max'], axis=1)

for each in ['pearson', 'spearman', 'kendall', correlation_ratio]:
    corr = data.corr(method=each)
    ax = sns.heatmap(
        corr, 
        vmin=-1, vmax=1, center=0,
        cmap=sns.diverging_palette(20, 220, n=200),
        square=True
    )
    ax.set_xticklabels(
        ax.get_xticklabels(),
        rotation=45,
        horizontalalignment='right'
    )
    try:
        ax.set_title(each.upper())
    except:
        ax.set_title(each.__name__.upper())
                 
    plt.show()

In [ ]:
# pd.plotting.scatter_matrix(data)
# STOPPED HERE - STOPPED HERE - STOPPED HERE - STOPPED HERE - STOPPED HERE - STOPPED HERE - STOPPED HERE - STOPPED HERE - STOPPED HERE - STOPPED HERE - STOPPED HERE - STOPPED HERE - STOPPED HERE - 

## SPECIFIC CASES (visual checking)

In [ ]:
df.groupby('failure').agg(num_unique_devices=('device','nunique'),
                          num_incidents=('device','count')
                          )

In [ ]:
#df.device.value_counts()

#df[df['failure']==1][['device', 'failure']].drop_duplicates()
devices_with_failure = df[df['failure']==1].device.unique().tolist()
print(devices_with_failure)


devices_without_failure = df[~df['device'].isin(devices_with_failure)].device.unique().tolist()
print(devices_without_failure)

In [ ]:
df[df['device'].isin(devices_with_failure)].sort_values(['device','date']).head()

In [ ]:
device = 'W1F0T034'

#device = 'W1F1230J'

print(len(df[df['device']==device].sort_values(['device','date'])))
df[df['device']==device].sort_values(['device','date']).tail(20)

In [ ]:
device='S1F0RRB1'

print(len(df[df['device']==device].sort_values(['device','date'])))
df[df['device']==device].sort_values(['device','date']).tail(50)

In [ ]:
df_test = df.copy()
df_test['7_8'] = df_test['attribute7'] == df_test['attribute8']
df_test['7_8'].value_counts()